In [12]:
from PIL import Image, ImageDraw, ImageFont
import os

def add_text_box_below_image(image_filename, text):
    # Load image
    image = Image.open(image_filename).convert("RGBA")
    width, height = image.size

    # Font setup (Arial from macOS system path)
    try:
        font = ImageFont.truetype("/System/Library/Fonts/Supplemental/Arial.ttf", 30)
    except IOError:
        raise Exception("Arial font not found. Adjust the path if necessary.")

    # Set padding and calculate text box size
    padding = 10
    dummy_draw = ImageDraw.Draw(image)
    text_width, text_height = dummy_draw.textsize(text, font=font)

    box_width = int(width * 1)
    box_height = text_height + 2 * padding
    new_height = height + box_height

    # Create new image with extended canvas
    new_image = Image.new('RGBA', (width, new_height), (255, 255, 255, 0))
    new_image.paste(image, (0, 0))

    # Position and draw box
    box_x0 = (width - box_width) // 2
    box_x1 = box_x0 + box_width

    # --- Sample background color from bottom-center pixel of original image
    background_sample = image.getpixel((width // 2, height - 1))  # RGBA

    # Make the sampled color semi-transparent
    background_with_alpha = background_sample[:3] + (220,)  # Add alpha

    # Draw the new box using the sampled color
    overlay = Image.new('RGBA', (width, box_height), (255, 255, 255, 0))
    draw = ImageDraw.Draw(overlay)
    draw.rectangle([box_x0, 0, box_x1, box_height], fill=background_with_alpha)

    # Draw text centered inside the new box
    text_x = (width - text_width) // 2
    text_y = padding
    draw.text((text_x, text_y), text, font=font, fill=(0, 0, 0, 255))

    # Paste the box onto the bottom of the new image
    new_image.paste(overlay, (0, height), overlay)

    # Save result as .jpg
    base_name = os.path.splitext(os.path.basename(image_filename))[0]
    output_filename = f"annotated_{base_name}.jpg"
    new_image.convert("RGB").save(output_filename, format="JPEG")
    print(f"Saved image with extended text box as '{output_filename}'.")

# Example usage
add_text_box_below_image("a.png", \
    "https://mobiledevmemo.com/mobile-ad-creative-how-to-produce-test-and-deploy-advertising-creative-at-scale/")


Saved image with extended text box as 'annotated_a.jpg'.
